In [ ]:
def DictionaryFromFile(filename):
    with open(filename) as f:
        everything = "".join(f.readlines()).split("\n")
        result = {}
        readline = 0
        key = None
        value = []
        while readline < len(everything):
            if len(everything[readline]) == 0:
                readline += 1
                continue
            if everything[readline][0] == '>':
                if (key != None):
                    result[key] = value
                key = everything[readline][1:]
                value = []
            else:
                numbers = everything[readline].split()
                added_value = []
                for n in numbers:
                    added_value.append(float(n))
                value.append(added_value)
            readline += 1
        if (key != None):
            result[key] = value
        return result 

#d = DictionaryFromFile("MA_PF00754_1_wrong.txt")
#print(d)


FileNotFoundError: ignored

In [ ]:
import torchvision
from torchvision.datasets import CIFAR10
import torch
torch.manual_seed(0)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PATH_OF_DATAtest= '/content/gdrive/"My Drive"/testprob'
!ls {PATH_OF_DATAtest}

ls: cannot access '/content/gdrive/My Drive/testprob': No such file or directory


In [ ]:
PATH_OF_DATAtrain= '/content/gdrive/"My Drive"/trainprob'
!ls {PATH_OF_DATAtrain}

ls: cannot access '/content/gdrive/My Drive/trainprob': No such file or directory


In [ ]:
PATH_OF_DATAtest= '/content/gdrive/"My Drive"/testprob'
!ls {PATH_OF_DATAtest}

ls: cannot access '/content/gdrive/My Drive/testprob': No such file or directory


In [ ]:
PATH_OF_DATAval= '/content/gdrive/"My Drive"/valprob'
!ls {PATH_OF_DATAval}

ls: cannot access '/content/gdrive/My Drive/valprob': No such file or directory


In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
class MyDataset(Dataset):
   def __init__(self, np_file_paths):
        #self.data = torch.FloatTensor(data.values.astype('float'))
        self.files = os.listdir(np_file_paths)
        self.np_file_paths = np_file_paths
   def __getitem__(self, index):
     x = np.load("{}/{}".format(self.np_file_paths, self.files[index]))
     x = torch.from_numpy(x).float()
     return x
   def __len__(self):
        return len(self.files)

        
#train_dataset = '/content/gdrive/"My Drive"/trainprob'
#test_dataset = '/content/gdrive/"My Drive"/testprob'


In [ ]:
train_dataset =  MyDataset('/content/drive/MyDrive/trainprob/')
test_dataset = MyDataset('/content/drive/MyDrive/testprob/')
val_dataset = MyDataset('/content/drive/MyDrive/valueprob/')

In [ ]:
train_dataset[12]

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 

In [ ]:
print(len(train_dataset))

2428


In [ ]:

trainloader = DataLoader(train_dataset, batch_size = 50)
valloader = DataLoader(val_dataset, batch_size = 50)
testloader = DataLoader(test_dataset, batch_size = 50)

In [ ]:
#у меня 22, а не 20
import torch
import torch.nn as nn
class _Model(torch.nn.Module):
    """A neural network model to predict phylogenetic trees."""

    def __init__(self):
        """Create a neural network model."""
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv1d(88, 88, 1, groups=22),
            torch.nn.BatchNorm1d(88),
            torch.nn.ReLU(),
            torch.nn.Conv1d(88, 32, 1),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AdaptiveAvgPool1d(1),
        )
        self.classifier = torch.nn.Linear(32, 3)

    def forward(self, x):
        x = x.view(x.size()[0], 88, -1)
        x = self.conv(x).squeeze(dim=2)
        return self.classifier(x)
class _ResidueModule(torch.nn.Module):

    def __init__(self, channel_count):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv1d(channel_count, channel_count, 1),
            torch.nn.BatchNorm1d(channel_count),
            torch.nn.ReLU(),
            torch.nn.Conv1d(channel_count, channel_count, 1),
            torch.nn.BatchNorm1d(channel_count),
            torch.nn.ReLU(),
        )

    def forward(self, x):
        return x + self.layers(x)

In [ ]:
#непонятно, как писать train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_epoch(model,
                optimizer,
                criterion,
                train_loader):
    loss_history = []
    for batch in train_loader: 
        optimizer.zero_grad()
        X_train, Y_train = batch # parse data
        X_train, Y_train = X_train.to(device), Y_train.to(device) # compute on gpu
        Y_pred = model(X_train) # get predictions
        loss = criterion(Y_pred, Y_train) # compute loss
        loss_history.append(loss.cpu().detach().numpy()) # write loss to log
        loss.backward()
        optimizer.step()
    return loss_history


In [ ]:
def validate(model,
             criterion,
             val_loader):
    cumloss = 0
    loss_history = []
    with torch.no_grad():
        for batch in val_loader:
            X_train, Y_train = batch # parse data
            X_train, Y_train = X_train.to(device), Y_train.to(device) # compute on gpu
            Y_pred = model(X_train) # get predictions
            loss = criterion(Y_pred, Y_train) # compute loss
            loss_history.append(loss.cpu().detach().numpy()) # write loss to log
            cumloss += loss
    return cumloss / len(val_loader), loss_history # mean loss and history

In [ ]:
from tqdm import tqdm

def train_model(model, optimizer, model_name=None, n_epochs=5):
  
    criterion = nn.CrossEntropyLoss().to(device)

    train_history = {}
    train_history['model_name'] = model_name
    train_history['loss_on_train'] = []
    train_history['loss_on_test'] = []

    for epoch in tqdm(range(n_epochs)):
        loss_on_train = train_epoch(model,
                                    optimizer,
                                    criterion,
                                    trainloader)
        _, loss_on_test = validate(model,
                                   criterion,
                                   testloader)
        train_history['loss_on_train'].extend(loss_on_train)
        train_history['loss_on_test'].extend(loss_on_test)
    return train_history

In [ ]:
import torch.optim as optim
model = _Model().to(device) 
optimizer = optim.SGD(model.parameters(), lr=0.001)

history = train_model(model, optimizer, model_name='n_layers2_sigmoid')


  0%|          | 0/5 [00:13<?, ?it/s]


ValueError: ignored

In [ ]:
#видимо, самой писать, а не из статьи
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change run time to gpu to fast training

model = _Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
loss_hist = [] # for plotting
fast_model = model
loss_function = torch.nn.CrossEntropyLoss(reduction='sum')
for epoch in range(1, num_epochs + 1):
            fast_model.train()
            sample_count, score = 0, 0.0
            optimizer.zero_grad()
            for i, (_a, x, _b, y) in enumerate(train_dataset):
                sample_count += x.size()[0]
                x = x.to(device, non_blocking=True)
                y = y.to(device, non_blocking=True)
                loss = loss_function(fast_model(x), y)
                loss.backward()
                score += float(loss)
                optimizer.step()
                optimizer.zero_grad()
            fast_model.eval()
            



RuntimeError: ignored